<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Preliminary-feature-selection" data-toc-modified-id="Preliminary-feature-selection-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Preliminary feature selection</a></span><ul class="toc-item"><li><span><a href="#Features-to-drop" data-toc-modified-id="Features-to-drop-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Features to drop</a></span></li><li><span><a href="#Features-to-use-(-Features-that-can-be-known-at-the-time-of-loan-issuance)" data-toc-modified-id="Features-to-use-(-Features-that-can-be-known-at-the-time-of-loan-issuance)-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Features to use ( Features that can be known at the time of loan issuance)</a></span></li></ul></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Data cleaning</a></span></li></ul></li></ul></div>

# Preprocessing

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)
sns.set(style = "whitegrid")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
warnings.filterwarnings(action="ignore")

## Load Data

In [73]:
accepted = pd.read_csv('accepted_2007_to_2018Q4.csv')

In [74]:
accepted.head(3)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [75]:
# Parse Years
accepted['year'] = pd.to_datetime(accepted.issue_d).dt.year
# Bin late loans into one group
accepted.loan_status = accepted.loan_status.apply(lambda x: np.where(x == 'Late (31-120 days)','Late',x))
accepted.loan_status = accepted.loan_status.apply(lambda x: np.where(x == 'In Grace Period','Late',x))
accepted.loan_status = accepted.loan_status.apply(lambda x: np.where(x == 'Late (16-30 days)','Late',x))
# Reduce the size of the dataset
accepted = accepted[(accepted.year.isin([2016,2017,2018]))& 
                    accepted.loan_status.isin(['Fully Paid','Charged Off','Late'])]

In [76]:
print(accepted.shape)

(550097, 152)


## Preliminary feature selection

In [77]:
accepted.reindex(sorted(accepted.columns), axis = 1).head()

,acc_now_delinq,acc_open_past_24mths,addr_state,all_util,annual_inc,annual_inc_joint,application_type,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,collection_recovery_fee,collections_12_mths_ex_med,debt_settlement_flag,debt_settlement_flag_date,deferral_term,delinq_2yrs,delinq_amnt,desc,disbursement_method,dti,dti_joint,earliest_cr_line,emp_length,emp_title,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,hardship_amount,hardship_dpd,hardship_end_date,hardship_flag,hardship_last_payment_amount,hardship_length,hardship_loan_status,hardship_payoff_balance_amount,hardship_reason,hardship_start_date,hardship_status,hardship_type,home_ownership,id,il_util,initial_list_status,inq_fi,inq_last_12m,inq_last_6mths,installment,int_rate,issue_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,last_pymnt_amnt,last_pymnt_d,loan_amnt,loan_status,max_bal_bc,member_id,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,mths_since_rcnt_il,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,next_pymnt_d,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,open_acc_6m,open_act_il,open_il_12m,open_il_24m,open_rv_12m,open_rv_24m,orig_projected_additional_accrued_interest,out_prncp,out_prncp_inv,payment_plan_start_date,pct_tl_nvr_dlq,percent_bc_gt_75,policy_code,pub_rec,pub_rec_bankruptcies,purpose,pymnt_plan,recoveries,revol_bal,revol_bal_joint,revol_util,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_earliest_cr_line,sec_app_fico_range_high,sec_app_fico_range_low,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_mths_since_last_major_derog,sec_app_num_rev_accts,sec_app_open_acc,sec_app_open_act_il,sec_app_revol_util,settlement_amount,settlement_date,settlement_percentage,settlement_status,settlement_term,sub_grade,tax_liens,term,title,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bal_il,total_bc_limit,total_cu_tl,total_il_high_credit_limit,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,total_rev_hi_lim,url,verification_status,verification_status_joint,year,zip_code
421101,0.0,3.0,WA,1.0,52000.0,NaN,Individual,25099.0,30359.0,0.5,0.0,0.0,0.0,N,NaN,NaN,0.0,0.0,NaN,Cash,0.58,NaN,Jan-1998,9 years,Scale Technician,764.0,760.0,3000.0,3000.0,A,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RENT,130956066,NaN,w,1.0,2.0,0.0,93.10,7.34,Mar-2018,Nov-2018,764.0,760.0,614.03,May-2018,3000.0,Fully Paid,141.0,NaN,132.0,242.0,18.0,7.0,4.0,26.0,NaN,NaN,7.0,18.0,NaN,7.0,NaN,NaN,0.0,1.0,1.0,4.0,15.0,7.0,6.0,19.0,1.0,7.0,0.0,0.0,0.0,1.0,7.0,0.0,0.0,1.0,2.0,0.0,1.0,NaN,0.00,0.00,NaN,96.7,0.0,1.0,0.0,0.0,major_purchase,n,0.0,141.0,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A4,0.0,36 months,Major purchase,0.0,150592.0,191216.0,30.0,141.0,0.0,30500.0,2.0,0.0,3011.577285,3011.58,11.58,0.0,3000.00,31000.0,https://lendingclub.com/browse/loanDetail.acti...,Source Verified,NaN,2018.0,988xx
421113,0.0,2.0,GA,47.0,55000.0,NaN,Individual,2222.0,10551.0,52.0,0.0,0.0,0.0,N,NaN,NaN,0.0,0.0,NaN,Cash,14.18,NaN,Aug-2001,10+ years,SDO Supervisor,679.0,675.0,5000.0,5000.0,B,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OWN,130968727,63.0,w,0.0,1.0,0.0,166.03,11.98,Mar-2018,Aug-2018,679.0,675.0,5019.97,Apr-2018,5000.0,Fully Paid,4829.0,NaN,77.0,199.0,3.0,3.0,0.0,74.0,74.0,82.0,33.0,3.0,NaN,12.0,NaN,NaN,1.0,3.0,3.0,4.0,7.0,6.0,13.0,18.0,3.0,14.0,0.0,0.0,0.0,2.0,14.0,1.0,1.0,0.0,0.0,2.0,2.0,NaN,0.00,0.00,NaN,95.7,33.3,1.0,1.0,1.0,other,n,0.0,11449.0,NaN,33.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B5,0.0,36 months,Other,0.0,28880.0,61551.0,24.0,28880.0,17431.0,22000.0,1.0,27751.0,5013.30666

### Features to drop

'acc_now_delinq','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','bc_util','chargeoff_within_12_mths','collection_recovery_fee','collections_12_mths_ex_med','debt_settlement_flag','debt_settlement_flag_date','deferral_term','delinq_2yrs','delinq_amnt','desc','disbursement_method','hardship_amount','hardship_dpd','hardship_end_date','hardship_flag','hardship_last_payment_amount','hardship_length','hardship_loan_status','hardship_payoff_balance_amount','hardship_reason','hardship_start_date','hardship_status','hardship_type','id','il_util','inq_fi','inq_last_12m','inq_last_6mths','last_credit_pull_d','last_fico_range_high','last_fico_range_low','last_pymnt_amnt','last_pymnt_d','max_bal_bc','member_id','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_last_delinq',
'mths_since_last_major_derog','mths_since_last_record','mths_since_rcnt_il','mths_since_recent_bc','mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','next_pymnt_d','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','open_acc_6m','open_act_il','open_il_12m','open_il_24m','open_rv_12m','open_rv_24m','orig_projected_additional_accrued_interest','out_prncp','out_prncp_inv','payment_plan_start_date','pct_tl_nvr_dlq','percent_bc_gt_75','policy_code','pub_rec','pub_rec_bankruptcies','recoveries','revol_bal','revol_bal_joint','revol_util','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med','sec_app_earliest_cr_line','sec_app_inq_last_6mths','sec_app_mths_since_last_major_derog','sec_app_num_rev_accts','sec_app_open_act_il','sec_app_revol_util','settlement_amount','settlement_date','settlement_percentage','settlement_status','settlement_term','tax_liens','tot_coll_amt','tot_cur_bal','tot_hi_cred_lim','total_acc','total_bal_ex_mort',
'total_bal_il','total_bc_limit','total_cu_tl','total_il_high_credit_limit','total_pymnt','total_pymnt_inv','total_rec_int','total_rec_late_fee','total_rec_prncp','total_rev_hi_lim','url'

### Features to use ( Features that can be known at the time of loan issuance)

In [78]:
accepted_new = accepted[['addr_state','all_util','annual_inc','annual_inc_joint','application_type',
'dti','dti_joint','earliest_cr_line','emp_length','fico_range_high',
'fico_range_low','funded_amnt','funded_amnt_inv','grade','home_ownership',
'initial_list_status','installment','int_rate','issue_d','year',
'loan_amnt','loan_status','mort_acc','purpose','pymnt_plan',
'sec_app_fico_range_high','sec_app_fico_range_low','sec_app_mort_acc',
'sec_app_open_acc','open_acc','sub_grade','term','verification_status',
'verification_status_joint']]

In [79]:
accepted_new.head(10)

,addr_state,all_util,annual_inc,annual_inc_joint,application_type,dti,dti_joint,earliest_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,home_ownership,initial_list_status,installment,int_rate,issue_d,year,loan_amnt,loan_status,mort_acc,purpose,pymnt_plan,sec_app_fico_range_high,sec_app_fico_range_low,sec_app_mort_acc,sec_app_open_acc,open_acc,sub_grade,term,verification_status,verification_status_joint
421101,WA,1.0,52000.0,NaN,Individual,0.58,NaN,Jan-1998,9 years,764.0,760.0,3000.0,3000.0,A,RENT,w,93.10,7.34,Mar-2018,2018.0,3000.0,Fully Paid,4.0,major_purchase,n,NaN,NaN,NaN,NaN,7.0,A4,36 months,Source Verified,NaN
421113,GA,47.0,55000.0,NaN,Individual,14.18,NaN,Aug-2001,10+ years,679.0,675.0,5000.0,5000.0,B,OWN,w,166.03,11.98,Mar-2018,2018.0,5000.0,Fully Paid,0.0,other,n,NaN,NaN,NaN,NaN,14.0,B5,36 months,Not Verified,NaN
421120,TX,90.0,40000.0,NaN,Individual,20.25,NaN,Mar-2007,< 1 year,699.0,695.0,7000.0,7000.0,B,MORTGAGE,w,232.44,11.98,Mar-2018,2018.0,7000.0,Fully Paid,0.0,home_improvement,n,NaN,NaN,NaN,NaN,13.0,B5,36 months,Verified,NaN
421121,NY,69.0,55000.0,NaN,Individual,26.63,NaN,Nov-2007,< 1 year,709.0,705.0,20300.0,20300.0,D,RENT,w,509.66,17.47,Mar-2018,2018.0,20300.0,Late,0.0,credit_card,n,NaN,NaN,NaN,NaN,10.0,D1,60 months,Verified,NaN
421135,FL,69.0,57000.0,NaN,Individual,27.58,NaN,Apr-2000,10+ years,684.0,680.0,30000.0,30000.0,D,OWN,w,1143.39,21.85,Mar-2018,2018.0,30000.0,Fully Paid,2.0,debt_consolidation,n,NaN,NaN,NaN,NaN,11.0,D5,36 months,Verified,NaN
421137,NY,34.0,85000.0,NaN,Individual,15.76,NaN,Nov-2008,10+ years,669.0,665.0,21000.0,21000.0,D,OWN,w,560.94,20.39,Mar-2018,2018.0,21000.0,Fully Paid,3.0,house,n,NaN,NaN,NaN,NaN,15.0,D4,60 months,Source Verified,NaN
421139,NY,21.0,150000.0,NaN,Individual,4.09,NaN,Aug-1996,10+ years,734.0,730.0,40000.0,40000.0,B,MORTGAGE,w,838.71,9.43,Mar-2018,2018.0,40000.0,Fully Paid,0.0,debt_consolidation,n,NaN,NaN,NaN,NaN,6.0,B1,60 months,Not Verified,NaN
421147,NJ,17.0,36610.0,NaN,Individual,13.85,NaN,Sep-2005,2 years,769.0,765.0,5000.0,5000.0,A,RENT,w,150.56,5.31,Mar-2018,2018.0,5000.0,Fully Paid,0.0,credit_card,n,NaN,NaN,NaN,NaN,15.0,A1,36 months,Not Verified,NaN
421165,CA,12.0,86000.0,NaN,Individual,9.66,NaN,Aug-2002,2 years,764.0,760.0,15000.0,15000.0,A,MORTGAGE,w,456.81,6.07,Mar-2018,2018.0,15000.0,Fully Paid,1.0,home_improvement,n,NaN,NaN,NaN,NaN,13.0,A2,36 months,Not Verified,NaN
421168,CA,65.0,38000.0,NaN,Individual,21.45,NaN,Apr-2002,5 years,664.0,660.0,12000.0,12000.0,C,RENT,w,410.54,14.07,Mar-2018,2018.0,12000.0,Fully Paid,0.0,debt_consolidation,n,NaN,NaN,NaN,NaN,9.0,C3,36 months,Source Verified,NaN


## Data cleaning

In [80]:
accepted_new['home_ownership'] = \
    accepted_new['home_ownership'].apply(lambda x: np.where((x == 'ANY') | (x == 'NONE'), 'OTHER', x))

In [81]:
# verification_status_joint

# dropped, because of too many missing values 
print(accepted_new.verification_status_joint.isnull().sum())
del accepted_new['verification_status_joint']
print(accepted_new.shape)

521178
(550097, 33)


In [82]:
# sec_app_fico_range_high

# dropped, because of too many missing values 

print(accepted_new.sec_app_fico_range_high.isnull().sum())
del accepted_new['sec_app_fico_range_high']
print(accepted_new.shape)

528011
(550097, 32)


In [83]:
# sec_app_fico_range_low

# dropped, because of too many missing values 

print(accepted_new.sec_app_fico_range_low.isnull().sum())
del accepted_new['sec_app_fico_range_low']
print(accepted_new.shape)

528011
(550097, 31)


In [84]:
# sec_app_mort_acc

# dropped, because of too many missing values 

print(accepted_new.sec_app_mort_acc.isnull().sum())
del accepted_new['sec_app_mort_acc']
print(accepted_new.shape)

528011
(550097, 30)


In [85]:
# 'sec_app_open_acc'

# dropped, because of too many missing values
print(accepted_new.sec_app_open_acc.isnull().sum())
del accepted_new['sec_app_open_acc']
print(accepted_new.shape)

528011
(550097, 29)


In [86]:
# purpose
accepted_new.purpose.value_counts()

debt_consolidation    311242
credit_card           110224
home_improvement       41369
other                  39425
major_purchase         13978
medical                 7814
car                     6393
small_business          6080
vacation                4641
moving                  4532
house                   4003
renewable_energy         393
wedding                    3
Name: purpose, dtype: int64

In [87]:
accepted_new['purpose'] = \
    accepted_new['purpose'].apply(lambda x: np.where((x == 'wedding') | (x == 'renewable_energy'), 'other', x))

accepted_new['purpose'] = \
    accepted_new['purpose'].apply(lambda x: np.where(x == 'house', 'home_improvement', x))

accepted_new.purpose.value_counts()

debt_consolidation    311242
credit_card           110224
home_improvement       45372
other                  39821
major_purchase         13978
medical                 7814
car                     6393
small_business          6080
vacation                4641
moving                  4532
Name: purpose, dtype: int64

In [88]:
# verification_status
print(accepted_new.shape)

accepted_new = pd.get_dummies(accepted_new, columns=['verification_status'])
print(accepted_new.shape)
accepted_new.head()

(550097, 29)
(550097, 31)


,addr_state,all_util,annual_inc,annual_inc_joint,application_type,dti,dti_joint,earliest_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,home_ownership,initial_list_status,installment,int_rate,issue_d,year,loan_amnt,loan_status,mort_acc,purpose,pymnt_plan,open_acc,sub_grade,term,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
421101,WA,1.0,52000.0,NaN,Individual,0.58,NaN,Jan-1998,9 years,764.0,760.0,3000.0,3000.0,A,RENT,w,93.10,7.34,Mar-2018,2018.0,3000.0,Fully Paid,4.0,major_purchase,n,7.0,A4,36 months,0,1,0
421113,GA,47.0,55000.0,NaN,Individual,14.18,NaN,Aug-2001,10+ years,679.0,675.0,5000.0,5000.0,B,OWN,w,166.03,11.98,Mar-2018,2018.0,5000.0,Fully Paid,0.0,other,n,14.0,B5,36 months,1,0,0
421120,TX,90.0,40000.0,NaN,Individual,20.25,NaN,Mar-2007,< 1 year,699.0,695.0,7000.0,7000.0,B,MORTGAGE,w,232.44,11.98,Mar-2018,2018.0,7000.0,Fully Paid,0.0,home_improvement,n,13.0,B5,36 months,0,0,1
421121,NY,69.0,55000.0,NaN,Individual,26.63,NaN,Nov-2007,< 1 year,709.0,705.0,20300.0,20300.0,D,RENT,w,509.66,17.47,Mar-2018,2018.0,20300.0,Late,0.0,credit_card,n,10.0,D1,60 months,0,0,1
421135,FL,69.0,57000.0,NaN,Individual,27.58,NaN,Apr-2000,10+ years,684.0,680.0,30000.0,30000.0,D,OWN,w,1143.39,21.85,Mar-2018,2018.0,30000.0,Fully Paid,2.0,debt_consolidation,n,11.0,D5,36 months,0,0,1


In [92]:
# term

print(accepted_new.term.value_counts())
accepted_new = pd.get_dummies(accepted_new,columns=['term'])
print(accepted_new.shape)
accepted_new.head()

 36 months    421937
 60 months    128160
Name: term, dtype: int64
(550097, 32)


,addr_state,all_util,annual_inc,annual_inc_joint,application_type,dti,dti_joint,earliest_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,grade,home_ownership,initial_list_status,installment,int_rate,issue_d,year,loan_amnt,loan_status,mort_acc,purpose,pymnt_plan,open_acc,sub_grade,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,term_ 36 months,term_ 60 months
421101,WA,1.0,52000.0,NaN,Individual,0.58,NaN,Jan-1998,9 years,764.0,760.0,3000.0,3000.0,A,RENT,w,93.10,7.34,Mar-2018,2018.0,3000.0,Fully Paid,4.0,major_purchase,n,7.0,A4,0,1,0,1,0
421113,GA,47.0,55000.0,NaN,Individual,14.18,NaN,Aug-2001,10+ years,679.0,675.0,5000.0,5000.0,B,OWN,w,166.03,11.98,Mar-2018,2018.0,5000.0,Fully Paid,0.0,other,n,14.0,B5,1,0,0,1,0
421120,TX,90.0,40000.0,NaN,Individual,20.25,NaN,Mar-2007,< 1 year,699.0,695.0,7000.0,7000.0,B,MORTGAGE,w,232.44,11.98,Mar-2018,2018.0,7000.0,Fully Paid,0.0,home_improvement,n,13.0,B5,0,0,1,1,0
421121,NY,69.0,55000.0,NaN,Individual,26.63,NaN,Nov-2007,< 1 year,709.0,705.0,20300.0,20300.0,D,RENT,w,509.66,17.47,Mar-2018,2018.0,20300.0,Late,0.0,credit_card,n,10.0,D1,0,0,1,0,1
421135,FL,69.0,57000.0,NaN,Individual,27.58,NaN,Apr-2000,10+ years,684.0,680.0,30000.0,30000.0,D,OWN,w,1143.39,21.85,Mar-2018,2018.0,30000.0,Fully Paid,2.0,debt_consolidation,n,11.0,D5,0,0,1,1,0


In [96]:
# sub_grade

accepted_new.sub_grade.value_counts()

B5    38025
C1    37951
C4    32972
C2    32870
B4    32634
C5    32324
C3    31463
B3    29867
B1    29544
B2    28343
A5    21296
A1    21031
D1    20102
D2    19178
A4    18595
D3    16732
A2    15035
A3    14963
D4    13946
D5    11876
E1     8268
E2     7055
E3     6663
E5     6630
E4     5944
F1     3792
F2     2541
F3     2233
F4     1885
F5     1742
G1     1413
G2      932
G3      790
G4      763
G5      699
Name: sub_grade, dtype: int64

In [95]:
# open_acc
accepted_new.open_acc.value_counts()

9.0     46900
10.0    45735
8.0     45152
11.0    42232
7.0     41705
12.0    37923
6.0     34889
13.0    33212
14.0    28832
5.0     25861
15.0    24452
16.0    20561
17.0    17180
4.0     16321
18.0    14397
19.0    11859
20.0     9635
21.0     7861
3.0      7823
22.0     6514
23.0     5456
24.0     4339
25.0     3510
26.0     2762
2.0      2527
27.0     2219
28.0     1833
29.0     1473
30.0     1256
31.0     1005
32.0      751
33.0      701
34.0      504
35.0      433
1.0       381
36.0      351
37.0      261
38.0      252
39.0      196
41.0      136
40.0      132
42.0      112
43.0       67
44.0       64
45.0       53
46.0       49
47.0       34
49.0       24
50.0       22
52.0       21
51.0       19
48.0       16
54.0       14
53.0       11
57.0        9
56.0        8
0.0         8
55.0        6
60.0        5
58.0        4
61.0        4
59.0        4
63.0        3
65.0        3
80.0        2
68.0        1
76.0        1
64.0        1
77.0        1
62.0        1
66.0        1
67.0  

In [89]:
# pymnt_plan - what's the description of this again? because y's sample size is too small - this column might be useless
print(accepted_new.pymnt_plan.value_counts())

n    549524
y       573
Name: pymnt_plan, dtype: int64
